<a href="https://colab.research.google.com/github/dishakhanted/Facial-analytics/blob/master/facial_analytics_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
import sys
sys.path.append("/content/drive/My Drive/Colab Notebooks/code")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install face_alignment

In [ ]:
use_cuda=True

In [ ]:
import numpy as np
import cv2,os
import dlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import torch.nn.functional as F
from PIL import Image
import datasets,hopenet, utils
from skimage import io
import face_alignment

In [ ]:
def landmark_detect(frame_name):
  fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False)
  #input = io.imread('/home/semantics/2.jpg')
  input = io.imread('frame_new.jpg')
  preds = fa.get_landmarks(input)
  preds = preds[0]
  return preds

In [ ]:
if __name__ == '__main__':
    cudnn.enabled = True
    batch_size = 1
    gpu = 0
    snapshot_path = '/content/drive/My Drive/Colab Notebooks/code/frame_new.jpg'
    out_dir = '/content/drive/My Drive/Colab Notebooks/code/output/video'
    video_path='/content/drive/My Drive/Colab Notebooks/code/input1.mp4'
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # ResNet50 structure
    model = hopenet.Hopenet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 66)
    face_model='/content/drive/My Drive/Colab Notebooks/code/mmod_human_face_detector.dat'
    # Dlib face detection model
  
    print ('Loading data.')

    transformations = transforms.Compose([transforms.Scale(224),
    transforms.CenterCrop(224), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    model.cuda(gpu)

    print ('Ready to test network.')

    # Test the Model
    model.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
    total = 0
    idx_tensor = [idx for idx in range(66)]
    idx_tensor = torch.FloatTensor(idx_tensor).cuda(gpu)
    video = cv2.VideoCapture(video_path)
    # New cv2
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))   # float
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)) # float

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter('/content/drive/My Drive/Colab Notebooks/code/videof.txt.avi', fourcc, 1, (width, height))


    txt_out = open('/content/drive/My Drive/Colab Notebooks/code/videof.txt', 'w')


    frame_num=1

    while frame_num <= 100:
        print ("frame num",frame_num)
    
        cv2.imwrite("frame_new.jpg", frame)
        ret,frame = video.read()
        if ret == False:
            break


        cv2_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        # Dlib detect
        dets = cnn_face_detector(cv2_frame, 1)

        for idx, det in enumerate(dets):
            # Get x_min, y_min, x_max, y_max, conf
            x_min = det.rect.left()
            y_min = det.rect.top()
            x_max = det.rect.right()
            y_max = det.rect.bottom()
            conf = det.confidence

            if conf > 1.0:
                print('yes')
                bbox_width = abs(x_max - x_min)
                bbox_height = abs(y_max - y_min)
                x_min -= 2 * bbox_width / 4
                x_max += 2 * bbox_width / 4
                y_min -= 3 * bbox_height / 4
                y_max += bbox_height / 4
                x_min = max(x_min, 0); y_min = max(y_min, 0)
                x_max = min(frame.shape[1], x_max); y_max = min(frame.shape[0], y_max)
                x_min=int(x_min)
                x_max=int(x_max)
                y_min=int(y_min)
                y_max=int(y_max)
                # Crop image
                img = cv2_frame[y_min:y_max,x_min:x_max]
                img = Image.fromarray(img)

                # Transform
                img = transformations(img)
                img_shape = img.size()
                img = img.view(1, img_shape[0], img_shape[1], img_shape[2])
                img = Variable(img).cuda(gpu)

                yaw, pitch, roll = model(img)

                yaw_predicted = F.softmax(yaw)
                pitch_predicted = F.softmax(pitch)
                roll_predicted = F.softmax(roll)
                # Get continuous predictions in degrees.
                yaw_predicted = torch.sum(yaw_predicted.data[0] * idx_tensor) * 3 - 99
                pitch_predicted = torch.sum(pitch_predicted.data[0] * idx_tensor) * 3 - 99
                roll_predicted = torch.sum(roll_predicted.data[0] * idx_tensor) * 3 - 99

                # Print new frame with cube and axis
                txt_out.write(str(frame_num) + ' %f %f %f\n' % (yaw_predicted, pitch_predicted, roll_predicted))
                utils.draw_axis(frame, yaw_predicted, pitch_predicted, roll_predicted, tdx = (x_min + x_max) / 2, tdy= (y_min + y_max) / 2, size = bbox_height/2)
                cv2.imwrite("/content/drive/My Drive/Colab Notebooks/code/output/output"+str(frame_num)+".jpg", frame)

              
                landmark_coordinates = landmark_detect(frame_num)
                res = {}
                res['frame_name']=str(frame_num)
                res['head_rotation']={}
                res['head_rotation']['yaw'] = float(yaw_predicted)
                res['head_rotation']['pitch'] = float(pitch_predicted)
                res['head_rotation']['roll'] = float(roll_predicted)
                res['facial_landmarks']= landmark_coordinates.tolist()
                print(res['facial_landmarks'])
                with open('./output/combined_'+str(frame_num)+'.json','w') as fitr:
                  json.dump(res,fitr)
          
        frame_num += 1
    out.release()
    video.release()

In [ ]:
model = hopenet.Hopenet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 66)
face_model='/content/drive/My Drive/Colab Notebooks/code/mmod_human_face_detector.dat'


In [ ]:
cnn_face_detector = dlib.cnn_face_detection_model_v1(face_model)
